In [1]:
import xarray as xr
import requests
from glob import glob
import pandas as pd
files = pd.Series(glob("hcast_nc/*.nc"))
files

0        hcast_nc/NI-030101_030630-PTWLEN.nc
1         hcast_nc/SI-990101_990630-HSIGN.nc
2          hcast_nc/NI-950101_950630-FSPR.nc
3           hcast_nc/SI-090101_090630-DIR.nc
4       hcast_nc/SI-930701_931231-PTWFRAC.nc
                        ...                 
2157     hcast_nc/SI-170101_171231-HSWELL.nc
2158       hcast_nc/SI-090701_091231-TM02.nc
2159    hcast_nc/SI-080101_080630-PTHSIGN.nc
2160     hcast_nc/NI-110101_110630-RTMM10.nc
2161      hcast_nc/NI-930701_931231-DEPTH.nc
Length: 2162, dtype: object

In [2]:
ds = xr.open_dataset(files[0])
ds

<xarray.Dataset> Size: 716MB
Dimensions:    (time: 1448, latitude: 103, longitude: 100)
Coordinates:
  * time       (time) datetime64[ns] 12kB 2003-01-01 ... 2003-06-30T21:00:00
  * latitude   (latitude) float32 412B -42.99 -42.9 -42.81 ... -33.56 -33.46
  * longitude  (longitude) float32 400B 170.4 170.5 170.6 ... 179.4 179.5 179.6
Data variables:
    WlPT01     (time, latitude, longitude) float64 119MB ...
    WlPT02     (time, latitude, longitude) float64 119MB ...
    WlPT03     (time, latitude, longitude) float64 119MB ...
    WlPT04     (time, latitude, longitude) float64 119MB ...
    WlPT05     (time, latitude, longitude) float64 119MB ...
    WlPT06     (time, latitude, longitude) float64 119MB ...

In [3]:
ds.time.to_pandas()

time
2003-01-01 00:00:00   2003-01-01 00:00:00
2003-01-01 03:00:00   2003-01-01 03:00:00
2003-01-01 06:00:00   2003-01-01 06:00:00
2003-01-01 09:00:00   2003-01-01 09:00:00
2003-01-01 12:00:00   2003-01-01 12:00:00
                              ...        
2003-06-30 09:00:00   2003-06-30 09:00:00
2003-06-30 12:00:00   2003-06-30 12:00:00
2003-06-30 15:00:00   2003-06-30 15:00:00
2003-06-30 18:00:00   2003-06-30 18:00:00
2003-06-30 21:00:00   2003-06-30 21:00:00
Name: time, Length: 1448, dtype: datetime64[ns]

In [4]:
str(ds.time.to_pandas().min().date()).replace("-", "")

'20030101'

In [9]:
min_time = ds.time.to_pandas().min()
gefs_min_time = pd.to_datetime("2000-01-08")
if min_time < gefs_min_time:
    min_time = gefs_min_time
date_str = str(min_time.date()).replace("-", "")
url = f"https://noaa-nws-gefswaves-reforecast-pds.s3.amazonaws.com/GEFSv12/reforecast/{min_time.year}/{date_str}/gridded/gefs.wave.{date_str}.c00.global.0p25.idx"
url

'https://noaa-nws-gefswaves-reforecast-pds.s3.amazonaws.com/GEFSv12/reforecast/2003/20030101/gridded/gefs.wave.20030101.c00.global.0p25.idx'

In [10]:
print(requests.get(url).text)

1:0:d=2003010103:WIND:surface:anl:
2:617696:d=2003010103:WDIR:surface:anl:
3:1505776:d=2003010103:UGRD:surface:anl:
4:2130789:d=2003010103:VGRD:surface:anl:
5:2746506:d=2003010103:HTSGW:surface:anl:
6:3132873:d=2003010103:IMWF:surface:anl:
7:3602324:d=2003010103:MWSPER:surface:anl:
8:4068646:d=2003010103:PERPW:surface:anl:
9:4565463:d=2003010103:WWSDIR:surface:anl:
10:5347394:d=2003010103:DIRPW:surface:anl:
11:6145064:d=2003010103:WVHGT:surface:anl:
12:6528379:d=2003010103:SWELL:1 in sequence:anl:
13:7017877:d=2003010103:SWELL:2 in sequence:anl:
14:7465980:d=2003010103:SWELL:3 in sequence:anl:
15:7860919:d=2003010103:WVPER:surface:anl:
16:8293238:d=2003010103:SWPER:1 in sequence:anl:
17:8875647:d=2003010103:SWPER:2 in sequence:anl:
18:9508668:d=2003010103:SWPER:3 in sequence:anl:
19:10123272:d=2003010103:WVDIR:surface:anl:
20:10776846:d=2003010103:SWDIR:1 in sequence:anl:
21:11690751:d=2003010103:SWDIR:2 in sequence:anl:
22:12632024:d=2003010103:SWDIR:3 in sequence:anl:
23:13513015:d=2